In [1]:
%load_ext cython

/Users/toddleonhardt/anaconda/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


# Dynamic arrays: allocate memory

In [2]:
n = 1000000

# Built-in Python arrays

from array import array
x_python = array('i', [0]*n)  # "i" means a C signed int
print('Python array item size: {} bytes'.format(x_python.itemsize))

# Numpy array

import numpy
x_numpy = numpy.zeros(n, dtype=numpy.intc)
print('Numpy array item size : {} bytes'.format(x_numpy.itemsize))

Python array item size: 4 bytes
Numpy array item size : 4 bytes


# Accept either via memory view

In [3]:
%%cython
# cython: language_level=3

cpdef memviewprops(int[:] x):
    print('{:10}:{}'.format('shape',x.shape))
    print('{:10}:{}'.format('strides', x.strides))
    print('{:10}:{}'.format('suboffsets', x.suboffsets))
    print('{:10}:{}'.format('ndim', x.ndim))
    print('{:10}:{}'.format('size', x.size))
    print('{:10}:{}'.format('itemsize', x.itemsize))
    print('{:10}:{}'.format('nbytes', x.nbytes))

In [4]:
print('Memory properties from a Python array')
print()
memviewprops(x_python)

print()
print('Memory properties from a Numpy array')
print()
memviewprops(x_numpy)

Memory properties from a Python array

shape     :[1000000, 0, 0, 0, 0, 0, 0, 0]
strides   :[4, 0, 0, 0, 0, 0, 0, 0]
suboffsets:[-1, 0, 0, 0, 0, 0, 0, 0]
ndim      :1
size      :1000000
itemsize  :4
nbytes    :4000000

Memory properties from a Numpy array

shape     :[1000000, 0, 0, 0, 0, 0, 0, 0]
strides   :[4, 0, 0, 0, 0, 0, 0, 0]
suboffsets:[-1, 0, 0, 0, 0, 0, 0, 0]
ndim      :1
size      :1000000
itemsize  :4
nbytes    :4000000


# Creating dynamic arrays with Cython itself

In [5]:
%%cython
# cython: language_level=3

from cython.view cimport array

cdef memviewprops(int[:] x):
    print('{:10}:{}'.format('shape',x.shape))
    print('{:10}:{}'.format('strides', x.strides))
    print('{:10}:{}'.format('suboffsets', x.suboffsets))
    print('{:10}:{}'.format('ndim', x.ndim))
    print('{:10}:{}'.format('size', x.size))
    print('{:10}:{}'.format('itemsize', x.itemsize))
    print('{:10}:{}'.format('nbytes', x.nbytes))
    return 
   
def cythonviewprops(n):
    x_cython = array(shape=(n,), itemsize=sizeof(int), format='i')
    memviewprops(x_cython)

In [6]:
cythonviewprops(n)

shape     :[1000000, 0, 0, 0, 0, 0, 0, 0]
strides   :[4, 0, 0, 0, 0, 0, 0, 0]
suboffsets:[-1, 0, 0, 0, 0, 0, 0, 0]
ndim      :1
size      :1000000
itemsize  :4
nbytes    :4000000


# Bonus: memview also works with static C arrays

In [7]:
%%cython
# cython: language_level=3

cdef memviewprops(int[:] x):
    print('{:10}:{}'.format('shape',x.shape))
    print('{:10}:{}'.format('strides', x.strides))
    print('{:10}:{}'.format('suboffsets', x.suboffsets))
    print('{:10}:{}'.format('ndim', x.ndim))
    print('{:10}:{}'.format('size', x.size))
    print('{:10}:{}'.format('itemsize', x.itemsize))
    print('{:10}:{}'.format('nbytes', x.nbytes))

def staticprops(n):
    cdef int x_static[1000000]
    memviewprops(x_static)

In [8]:
staticprops(n)

shape     :[1000000, 0, 0, 0, 0, 0, 0, 0]
strides   :[4, 0, 0, 0, 0, 0, 0, 0]
suboffsets:[-1, 0, 0, 0, 0, 0, 0, 0]
ndim      :1
size      :1000000
itemsize  :4
nbytes    :4000000


# Memory views work with slices

In [10]:
%%cython

cdef f(int[:,:] view, int num):
    view[-2:,-2:] = num
    
def main():
    import numpy
    x = numpy.zeros(shape=(4, 4), dtype=numpy.intc)
    f(x, 1)
    print(x)
    
main()

[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]]


## Slicing makes copying easier

In [11]:
%%cython
# cython: language_level=3

cdef copy(double[:] x, double[:] out):
    " Copy x values into out "
    out[:] = x
    
def main():
    import numpy
    import array
    # Empty Python array
    py_array = array.array('d', [0]) * 3
    # Numpy array: random values
    npy_array = numpy.random.rand(3) # Random values
    # Call function
    copy(npy_array, py_array)
    print(py_array)
    
main()

array('d', [0.2922741845358776, 0.060398340834783903, 0.969681628371355])


# And now for something completely different

### Array creation, which is faster?

In [12]:
def f1(n):
    return array('i', [0]*n)  # Big list, then array
    
def f2(n):
    return array('i', [0])*n  # Small list, expand array

def f3(n):
    return numpy.zeros(n, dtype=numpy.intc)

In [13]:
%timeit f1(10000000)
%timeit f2(10000000)
%timeit f3(10000000)

1 loop, best of 3: 378 ms per loop
100 loops, best of 3: 9.72 ms per loop
100 loops, best of 3: 7.83 ms per loop
